In [ ]:
#pip install -U langchain-community
#pip install panel
#pip install pypdf 
#pip install chromadb
#%pip install --upgrade --quiet  docx2txt

In [ ]:
import panel as pn
import param
import pandas as pd
from langchain.document_loaders import PyPDFLoader, CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain_community.document_loaders import Docx2txtLoader

In [ ]:
# Define the helper function to load the database
def load_db(file, file_type, chain_type, k):
    # Load documents based on file type
    #can load 3 types of documents pdf,docx,csv
 

   
    <code>
    if file_type == 'pdf':
        loader = PyPDFLoader(file)
        documents = loader.load()
    elif file_type == 'csv':
        loader = CSVLoader(file_path=file)
        documents = loader.load()
    elif file_type == 'docx':
        loader = Docx2txtLoader(file)
        documents = loader.load()
    else:
        raise ValueError(f"Unsupported file type: {file_type}")
    </code>
   

In [ ]:
 

    # Split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    
    # Define embeddings
    embeddings = OpenAIEmbeddings(openai_api_key='MY_OPENAI_KEY')
    
    # Create vector database from data
    db = Chroma.from_documents(docs, embeddings)
    
    # Define retriever
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    
    # Create a chatbot chain
    llm_model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key='MY_OPENAI_KEY')
    qa = ConversationalRetrievalChain.from_llm(
        llm=llm_model, 
        chain_type=chain_type, 
        retriever=retriever
    )
    return qa

In [ ]:
import param
import panel as pn

class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    loaded_file = param.String("/Users/pradhiksha.suresh/OneDrive - Incedo Technology Solutions Ltd/Desktop/Python/Space.pdf")
    file_type = param.String("pdf")

    def __init__(self, **params):
        super(cbfs, self).__init__(**params)
        self.panels = []
        self.qa = load_db(self.loaded_file, self.file_type, "stuff", 4)

    def call_load_db(self, event):
        if file_input.value:
            with open("temp_file", "wb") as f:
                f.write(file_input.value)
            self.loaded_file = "temp_file"
            self.file_type = file_input.filename.split('.')[-1]
            self.qa = load_db(self.loaded_file, self.file_type, "stuff", 4)
            self.clr_history()
            return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")
        else:
            return pn.pane.Markdown(f"Loaded Default File: {self.loaded_file}")

    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=600)), scroll=True)
        
        result = self.qa({"question": query, "chat_history": self.chat_history})
        self.chat_history.append((query, result["answer"]))
        self.answer = result['answer'] 
        
        self.panels.append(
            pn.Row('User:', pn.pane.Markdown(query, width=600)),
        )
        self.panels.append(
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=600, styles={'background-color': '#F6F6F6'}))
        )
        
        inp.value = ''  # clears the input box
        return pn.WidgetBox(*self.panels, scroll=True)

    def clr_history(self, event=None):
        self.chat_history = []
        self.panels = []  # Clear the panels as well
        return pn.pane.Markdown("Chat history cleared.")

cb = cbfs()

file_input = pn.widgets.FileInput(accept='.pdf,.csv,.docx')
button_load = pn.widgets.Button(name="Load DB", button_type='primary')
button_load.on_click(cb.call_load_db)
button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning')
button_clearhistory.on_click(cb.clr_history)
inp = pn.widgets.TextInput(placeholder='Enter text here…')

# Bind the convchain method to the input field
conversation = pn.bind(cb.convchain, inp)


# Layout including the file input and load button
tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    conversation,
    pn.layout.Divider(),
)

tab2 = pn.Column(
    pn.Row(file_input, button_load),
    pn.layout.Divider(),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# ChatBot')),
    pn.Tabs(('Conversation', tab1)),
    pn.Row(button_clearhistory),
    tab2
)

dashboard.servable()